In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Datasets/Dog/Dog conditions dataset-1.csv",encoding='cp1252')

In [ ]:
df.head()

,Question,Answer
0,How to spot for the signs of Fleas?,Every pet owner should be aware of the signs o...
1,How to get rid of fleas on your pet?,"The scratching, the biting at the fur, maybe e..."
2,How to control fleas and ticks outside?,"Mow Grass, Trim Shrubs, and Get Rid of Debris\..."
3,How common areÿfood allergiesÿin dogs?,Ten percent of all allergy cases in dogs are f...
4,What are the common signs of a food allergy?,"Anything from chronic ear inflammation, gastro..."


### Data Pre-Processing

In [ ]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):

    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)

    if stopwords:
         sentence = remove_stopwords(sentence)

    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word)
    #sentence=sent_stemmed

    return sentence

def get_cleaned_sentences(df,stopwords=False):
    sents=df[["Question"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["Question"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['spot signs fleas', 'rid fleas pet', 'control fleas ticks outside', 'common arefood allergiesin dogs', 'common signs food allergy', 'common things trigger food allergy dog', 'causes foodallergies', 'determine dog hasfoodallergies causing problem', 'changing dogs diet trigger food allergy', 'dog suddenly develop food allergy eaten years problems happening', 'treat dog food allergies', 'cook dog buy food raw diet help', 'mypuppyfrom developing food allergies', 'allergic dermatitis', 'yeast infection dog', 'folliculitis', 'impetigo', 'seborrhea', 'ringworm dogs', 'shedding hair loss alopecia', 'mange mites', 'fleas dog', 'ticks', 'skin colour change texture change indicates', 'dry flaky skin dogs', 'acral lick granuloma', 'skin tumors dogs', 'hot spots dogs', 'immune disorders dogs', 'anal sac disease dogs', 'vet', 'signs tickborne diseases', 'tick concerns dogs', 'skin treatments dogs safe', 'use fogger', 'symptons anal sac disease dogs', 'stages anal sac disease', 'preventions anal sac

### Answer Retreival using BOW(Bag of words)

In [ ]:
#vector representation for each sentence using BOW, and then computing the distance between two vectors by taking the cosine similarity
import numpy
import pprint
sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
#for key, value in dictionary.items():
    #print(key, ' : ', value)
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    #print(sent)
    #print(embedding)
    question_orig="How to get rid of fleas on your pet?"
    question=clean_sentence(question_orig,stopwords=False);
    question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)



 how to get rid of fleas on your pet 
 [(0, 1), (2, 1), (3, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    global context
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        #print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;

    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0])
    context = FAQdf.iloc[index_sim,1]
    print(context)

retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences)

Question:  how to get rid of fleas on your pet


Retrieved:  How to spot for the signs of Fleas?
Every pet owner should be aware of the signs of a possible flea infestation. They include:

Your dog (or cat) is scratching. Even if you don?t catch fleas red-handed, if you see your pet scratching or biting at its fur, fleas may well be the culprit. That?s because not only can fleas cause a sharp pain when they bite, their salivary glands give off a substance that's irritating to many dogs and cats.

You can see them. Adult fleas are about an eighth of an inch long. They?re reddish-brown and very thin. It?s hard to really see what they look like without a microscope (though it?s easier on a light-coloured fur), but they do have big back legs. They can jump, by some measurements, upward and outward at least 12 inches in a single leap. And one estimation finds that for every adult flea found on your pet, there are at least 100 immature ones hanging around.

You can see what they leave behind

### Using Word2Vec

In [ ]:
#Word2Vec embedding

from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved Word2vec model")

w2vec_embedding_size=len(v2w_model['computer']);

[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved Word2vec model


In [ ]:
# Finding Phrase Embeddings from Word Embeddings

def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):

        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [ ]:
#With w2Vec

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

Question:  how to get rid of fleas on your pet


Retrieved:  How to get rid of fleas on your pet?
The scratching, the biting at the fur, maybe even the big patches of irritated skin all point to fleas. A fine-toothed flea comb run through your pet?s fur can confirm it if it picks up adult fleas and maybe some ?flea dirt,? little spots that look like pepper.
The "dirt" is actually flea poop, made mostly from your pet?s dried blood. You may be able to see fleas without a comb if you separate the hair between your pet?s shoulder blades or at the base of the tail.
First, check with your veterinarian.
A lot of those flea prevention products -- as long as they?re not combined with heartworm prevention -- you can get them over the counter.
The soap in a bath can act as a gentle insecticide. Other problems that come from fleabites -- open sores from scratching or other skin problems like rashes -- may call for stronger kinds of help. Again, see your vet.
If you?re starting to see some irritati